In [1]:
import os
import numpy as np
import requests
import re
import copy
import networkx as nx

# Get Input

In [2]:
sessionId = os.environ["ADVENT_OF_CODE_SESSION_ID"]

In [3]:
r = requests.get("https://adventofcode.com/2018/day/16/input", cookies={"session": sessionId}, verify=False)

/Users/cmartin/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
input = r.content

In [28]:
lines = [line.decode('utf-8') for line in input.splitlines()]

In [29]:
a = "[3,5]"

In [30]:
list(a)

['[', '3', ',', '5', ']']

In [32]:
lines[0]

'Before: [1, 1, 2, 0]'

In [33]:
[int(n) for n in lines[0][9:-1].split(',')]

[1, 1, 2, 0]

In [44]:
ops = []
for i in range(0, len(lines), 4):
    if 'Before' not in lines[i]:
        break
    op = {'before': [int(n) for n in lines[i][9:-1].split(',')],
          'op': [int(n) for n in lines[i+1].split(' ')],
          'after': [int(n) for n in lines[i+2][9:-1].split(',')]}
    ops.append(op)
    

In [45]:
ops[0]

{'before': [1, 1, 2, 0], 'op': [8, 1, 0, 3], 'after': [1, 1, 2, 1]}

# Setup

In [48]:
def addr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] + registers[B]
    return registers
                          
def addi(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] + B
    return registers

def mulr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] * registers[B]
    return registers

def muli(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] * B
    return registers

def banr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] & registers[B]
    return registers

def bani(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] & B
    return registers

def borr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] | registers[B]
    return registers

def bori(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A] | B
    return registers

def setr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = registers[A]
    return registers

def seti(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = A
    return registers

def gtir(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if A > registers[B] else 0
    return registers

def gtri(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if registers[A] > B else 0
    return registers

def gtrr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if registers[A] > registers[B] else 0
    return registers

def eqir(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if A ==registers[B] else 0
    return registers

def eqri(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if registers[A] == B else 0
    return registers

def eqrr(registers_in, A, B, C):
    registers = copy.copy(registers_in)
    registers[C] = 1 if registers[A] == registers[B] else 0
    return registers

instructions = [addr,
                 addi,
                 mulr, 
                 muli,
                 banr, 
                 bani, 
                 borr, 
                 bori, 
                 setr, 
                 seti, 
                 gtir, 
                 gtri, 
                 gtrr, 
                 eqir, 
                 eqri, 
                 eqrr]

# Test Case

In [68]:
op = {'before': [3, 2, 1, 1],
     'op': [9,2,1,2],
     'after': [3, 2, 2, 1]}

In [70]:
count = 0
before = op['before']
after = op['after']
A, B, C = op['op'][1:4]
for f in instructions:
    try:
        result = f(before, A,B,C)
        if result == after:
            print(f.__name__)
            count+=1
    except:
        continue
count       

addi
mulr
seti


3

# Part 1

In [73]:
three_poss = 0

for op in ops:
    count = 0
    before = op['before']
    after = op['after']
    A, B, C = op['op'][1:4]
    for f in instructions:
        try:
            result = f(before, A,B,C)
            if result == after:
                count+=1
        except:
            continue
    if count >= 3:
        three_poss+=1
three_poss


614

# Part 2

In [81]:
from collections import Counter

In [82]:
Counter([a['op'][0] for a in ops])

Counter({8: 48,
         5: 58,
         7: 52,
         2: 47,
         3: 48,
         9: 57,
         1: 46,
         14: 59,
         4: 44,
         12: 43,
         13: 56,
         11: 43,
         15: 55,
         0: 48,
         10: 48,
         6: 57})

In [85]:
len(ops)

809

In [113]:
poss_instructions = {i: set(instructions) for i in range(16)}

for op in ops:
    before = op['before']
    after = op['after']
    opcode, A, B, C = op['op']
    to_remove = set()
    for f in poss_instructions[opcode]:
        try:
            result = f(before, A,B,C)
            if result != after:
                to_remove.add(f)
        except:
            continue
    for f in to_remove:
        poss_instructions[opcode].remove(f)


In [114]:
known_opcodes = {}
while poss_instructions:
    single = min(poss_instructions.keys(), key=lambda k: len(poss_instructions[k]))
    if len( poss_instructions[single]) > 1:
        break
    op = poss_instructions[single].pop()
    poss_instructions.pop(single)
    
    known_opcodes[single] = op
    
    for k, v in poss_instructions.items():
        if op in v:
            v.remove(op)
    
    

In [115]:
known_opcodes

{10: <function __main__.eqrr(registers_in, A, B, C)>,
 9: <function __main__.eqri(registers_in, A, B, C)>,
 0: <function __main__.eqir(registers_in, A, B, C)>,
 3: <function __main__.gtri(registers_in, A, B, C)>,
 13: <function __main__.gtrr(registers_in, A, B, C)>,
 5: <function __main__.gtir(registers_in, A, B, C)>,
 7: <function __main__.banr(registers_in, A, B, C)>,
 11: <function __main__.bani(registers_in, A, B, C)>,
 12: <function __main__.setr(registers_in, A, B, C)>,
 15: <function __main__.seti(registers_in, A, B, C)>,
 2: <function __main__.addr(registers_in, A, B, C)>,
 6: <function __main__.mulr(registers_in, A, B, C)>,
 1: <function __main__.borr(registers_in, A, B, C)>,
 14: <function __main__.addi(registers_in, A, B, C)>,
 8: <function __main__.bori(registers_in, A, B, C)>,
 4: <function __main__.muli(registers_in, A, B, C)>}

In [122]:
list(reversed(lines)).index('')

1023

In [126]:
operations = []
for line in lines[-1023:]:
    operations.append([int(n) for n in line.split(' ')])

In [128]:
register = [0, 0, 0, 0]
for op in operations:
    opcode, A ,B, C = op
    
    register = known_opcodes[opcode](register, A, B, C)

In [129]:
register

[656, 656, 2, 4]